I'm trying to find the most efficient way to vectorie my importance sampling calculation. Since my other notebook is actually running, I'll mess with it here. 

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
colors = sns.color_palette()
import numpy as np
np.random.seed(0)#"Random"

In [4]:
#First, will need the parameters
redshift = .9 #one redshift, for now. 
nSamples = 1000
vals = {}
z = [0.23, 1.5]
vals['Mp'] = [2.0e14, 1.0e14]
vals['A'] = [1.944, 0.293]
vals['B1'] = [1.97, 3.07]
vals['B2'] = [0.7, 1.2]
vals['B3'] = [0.40, 0.73]

#interpolates naively between the points given in the original paper
z_params = {}
from scipy.stats import linregress
for key, val in vals.iteritems():
    slope, intercept, r, p, stderr = linregress(z, val)
    z_params[key] = (slope, intercept)

#return the values of the parameters at a given z. 
getMassParams = lambda z : {key:val[0]*z+val[1] for key,val in z_params.iteritems()}

def n_approx(m,z):
    params = getMassParams(z)
    return params['A']*np.exp(-params['B1']*(m/params['Mp']) \
                    -0.5*params['B2']*(m/params['Mp'])**2 \
                    -0.166*params['B3']*(m/params['Mp'])**3)

def log_n_approx(m,z):
    params = getMassParams(z)
    return -1*(params['B1']*(m/params['Mp']) \
                    +0.5*params['B2']*(m/params['Mp'])**2 \
                    +0.166*params['B3']*(m/params['Mp'])**3)+np.log(params['A'])

In [5]:
#use MCMC to sample masses from the mass function
import emcee as mc
ndim = 1
nWalkers = 1000
nSteps = 500
nBurn = 400
nCores = 1

Mmin, Mmax = 1e13, 5e15
pos0 = np.random.uniform(np.log10(Mmin), np.log10(Mmax), size = nWalkers).reshape((nWalkers, ndim))

def log_p(logM, z):
    logM = logM[0]
    m = 10**logM
    if m>Mmax or m<Mmin:
        return -np.inf
    
    return log_n_approx(m, z)

sampler = mc.EnsembleSampler(nWalkers, ndim, log_p, args=[redshift],threads = nCores)

In [6]:
sampler.run_mcmc(pos0, nSteps);

In [7]:
logMSamples = sampler.chain[:,nBurn:, :].reshape((-1, nSamples))[0,:]
mSamples = 10**logMSamples

In [8]:
M_piv = 2.35e14
logL0_true = 3.141
a_true, b_true = 0.842, -0.03
B_l_true = 0.642
sigma_l_true = 0.184

_A_lam = lambda a,b,z : a*pow((1+z)/1.3, b)

from scipy.stats import norm
from scipy.stats import multivariate_normal as mv_norm
#forward model
def logLam(logLam0, a, b, B_lam, z, M):
    A_lam = _A_lam(a,b,z)
    return logLam0+A_lam*np.log(M/M_piv)+B_lam*np.log((1+z)/1.3)

def logLamSample(logLam0, a, b, B_lam,sigma_lam, z,M):
    return norm.rvs(loc = logLam(logLam0, a, b, B_lam, z, M), scale = sigma_lam, size = M.shape[0])

In [9]:
logRichness = logLamSample(logL0_true, a_true, b_true, B_l_true, sigma_l_true, redshift, mSamples)

In [10]:
from scipy.misc import logsumexp
from scipy.stats import norm, multivariate_normal as mv_norm
#returns a logmass given a richness
def invLogLam(logLam0, a, b, B_lam, z, logRich):
    A_lam = _A_lam(a,b,z)
    return np.log(M_piv)+(logRich-logLam0-B_lam*np.log((1+z)/1.3))/A_lam
#TODO Make this one synonymous with it non-inverse version
#TODO Consider checking if logRich is a vector or a number, and acting accodingly. 
def invLogLamSample(logLam0, a, b, B_lam,sigma_mass, z,logRich, size = 100):
    mu = invLogLam(logLam0, a, b, B_lam, z, logRich)
    return mv_norm.rvs(mean = mu, cov = sigma_mass*np.identity(logRich.shape[0]), size =  size).T#(logRich.shape[0], size)
    #return norm.rvs(loc = mu, scale = sigma_mass, size =  size)#( size,)

In [29]:
def log_liklihood(logL0, a,b, B_l, sigma, z, logRich):
    sigma_mass = 1 #TODO idk what this should be; i suppose it's my discretion, or some relation to
    logL = 0
    #TODO Vectorize furhter?
    '''
    for lr in logRich:
        #use truths as really really good guess. Can relax later. 
        massSamples = invLogLamSample(logL0_true, a_true, b_true, B_l_true,sigma_mass, z, lr)

        logPRich = norm.logpdf(lr, loc =logLam(logL0, a, b, B_l, z, massSamples), scale = sigma)
        logPMass = log_n_approx(massSamples,z)
        logPMass_sample = norm.logpdf(massSamples, loc = invLogLam(logL0_true, a_true, b_true, B_l_true, z, lr), scale = sigma_mass)
        
        logL+= logsumexp(logPRich+logPMass-logPMass_sample)
        
    return logL
    
    '''
    logRichArr = np.array([logRich[:] for i in xrange(100)]).T
    massSamples = invLogLamSample(logL0_true, a_true, b_true, B_l_true,sigma_mass, z, logRich)
    logPRich = norm.logpdf(logRichArr, loc =logLam(logL0, a, b, B_l, z, massSamples), scale = sigma)
    logPMass = log_n_approx(massSamples,z)
    logPMass_sample = norm.logpdf(massSamples, loc = invLogLam(logL0_true, a_true, b_true, B_l_true, z, logRichArr), scale = sigma_mass)
    return np.sum(logsumexp(logPRich+logPMass-logPMass_sample, axis = 1), axis = 0)

In [30]:
%%timeit
print log_liklihood(logL0_true, a_true, b_true, B_l_true, sigma_l_true, redshift, logRichness)

-7748103.00724
-7747713.54315
-7747958.08842
-7748346.86672
1 loops, best of 3: 3.17 s per loop


In [33]:
def invLogLamSample2(logLam0, a, b, B_lam,sigma_mass, z,logRich, size = 100):
    mu = invLogLam(logLam0, a, b, B_lam, z, logRich)
    #return mv_norm.rvs(mean = mu, cov = sigma_mass*np.identity(logRich.shape[0]), size =  size).T#(logRich.shape[0], size)
    return norm.rvs(loc = mu, scale = sigma_mass, size =  size)#( size,)

def log_liklihood2(logL0, a,b, B_l, sigma, z, logRich):
    sigma_mass = 1 #TODO idk what this should be; i suppose it's my discretion, or some relation to
    logL = 0

    for lr in logRich:
        #use truths as really really good guess. Can relax later. 
        massSamples = invLogLamSample2(logL0_true, a_true, b_true, B_l_true,sigma_mass, z, lr)

        logPRich = norm.logpdf(lr, loc =logLam(logL0, a, b, B_l, z, massSamples), scale = sigma)
        logPMass = log_n_approx(massSamples,z)
        logPMass_sample = norm.logpdf(massSamples, loc = invLogLam(logL0_true, a_true, b_true, B_l_true, z, lr), scale = sigma_mass)
        
        logL+= logsumexp(logPRich+logPMass-logPMass_sample)
        
    return logL

In [34]:
%%timeit
print log_liklihood2(logL0_true, a_true, b_true, B_l_true, sigma_l_true, redshift, logRichness)

-7748171.73712
-7748121.49615
-7748122.11479
-7748144.94552
1 loops, best of 3: 842 ms per loop
